In [ ]:
# pip install ipynb

In [ ]:
# from ipynb.fs.full.myAdmin import create_connection, view_course_details, select_all_asgnmts, select_all_videos

In [1]:
import sqlite3
from sqlite3 import Error
import datetime
import pandas as pd

In [2]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

In [3]:
def view_course_details(conn):
    cur = conn.cursor()
    
    sqlstr = "SELECT * FROM Course"
    details = {'iD':[], 'name':[], 'prof':[]}
    for row in cur.execute(sqlstr):
        details['iD'].append(row[0])
        details['name'].append(row[1])
        details['prof'].append(row[2])
    return details

In [7]:
def select_all_asgnmts(conn, course_num):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    details = {'assID':[], 'status':[], 'dep':[], 'deadline':[]}
    for row in cur.execute("SELECT * FROM Ass"+str(course_num)):
        details['assID'].append(row[0])
        details['status'].append(row[1])
        details['dep'].append(row[2])
        row3 = row[3].split('-')
        d = int(row3[2].split(' ')[0])
        m = int(row3[1])
        y = int(row3[0])
        date = datetime.datetime(y,m,d)
        details['deadline'].append(date)
    return details

In [8]:
database = r"mydb.db"
conn = create_connection(database)

In [9]:
select_all_asgnmts(conn, 1)

{'assID': [1, 2],
 'status': ['No', 'No'],
 'dep': [2, 4],
 'deadline': [datetime.datetime(2020, 9, 10, 0, 0),
  datetime.datetime(2020, 9, 11, 0, 0)]}

In [10]:
def select_all_videos(conn, course_num):
    cur = conn.cursor()
    details = {'ID':[], 'status':[]}
    for row in cur.execute("SELECT * FROM Vid"+str(course_num)):
        details['ID'].append(row[0])
        details['status'].append(row[1])
    return details

In [11]:
courses = view_course_details(conn)

In [12]:
cdf = pd.DataFrame(courses, columns=['iD','name','prof'])

In [13]:
cdf

,iD,name,prof
0,1,S1,Mr. A
1,2,S2,Mrs. B
2,3,S3,Ms. C
3,4,S4,Mr. D


In [14]:
course1 = select_all_videos(conn, 1)
course2 = select_all_videos(conn, 2)
course3 = select_all_videos(conn, 3)
course4 = select_all_videos(conn, 4)

In [15]:
cdf['videos'] = [course1, course2, course3, course4]

In [16]:
ass1 = select_all_asgnmts(conn, 1)
ass2 = select_all_asgnmts(conn, 2)
ass3 = select_all_asgnmts(conn, 3)
ass4 = select_all_asgnmts(conn, 4)

In [17]:
a1 = pd.DataFrame(ass1, columns=['assID','status','dep', 'deadline'])
a2 = pd.DataFrame(ass2, columns=['assID','status','dep', 'deadline'])
a3 = pd.DataFrame(ass3, columns=['assID','status','dep', 'deadline'])
a4 = pd.DataFrame(ass4, columns=['assID','status','dep', 'deadline'])

cdf['assgnmts'] = [ass1, ass2, ass3, ass4]

In [18]:
x = datetime.datetime.now()
print(x)

2020-09-05 21:50:58.756235


In [19]:
stack1 = []
for each in a1['assID']:
    if a1.iloc[each-1]['status']=='No':
        stack1.append((a1.iloc[each-1]['deadline'] - x).days)
stack2 = []
for each in a2['assID']:
    if a2.iloc[each-1]['status']=='No':
        stack2.append((a2.iloc[each-1]['deadline'] - x).days)
stack3 = []
for each in a3['assID']:
    if a3.iloc[each-1]['status']=='No':
        stack3.append((a3.iloc[each-1]['deadline'] - x).days)
stack4 = []
for each in a4['assID']:
    if a4.iloc[each-1]['status']=='No':
        stack4.append((a4.iloc[each-1]['deadline'] - x).days)

print('Course1',stack1)
print('Course2',stack2)
print('Course3',stack3)
print('Course4',stack4)

Course1 [4, 5]
Course2 []
Course3 []
Course4 []


In [40]:
workstack = []

if x.strftime("%A") in ['Sunday']:
    # search late pending works
    for i in range(len(stack1)):
        if stack1[i] <= 0:
            workstack.append((1,a1.iloc[i]['dep'],stack1[i],i+1))
    for i in range(len(stack2)):
        if stack2[i] <= 0:
            workstack.append((2,a2.iloc[i]['dep'],stack2[i],i+1))
    for i in range(len(stack3)):
        if stack3[i] <= 0:
            workstack.append((3,a3.iloc[i]['dep'],stack3[i],i+1))
    for i in range(len(stack4)):
        if stack4[i] <= 0:
            workstack.append((4,a4.iloc[i]['dep'],stack4[i],i+1))
                
else:
    count = 0
    size = stack1 + stack2 + stack3 + stack4
    for _ in range(max(size)+1):
        for i in range(len(stack1)):
            if stack1[i] == count:
                workstack.append((1,a1.iloc[i]['dep'],stack1[i],i+1))
        for i in range(len(stack2)):
            if stack2[i] == count:
                workstack.append((2,a2.iloc[i]['dep'],stack2[i],i+1))
        for i in range(len(stack3)):
            if stack3[i] == count:
                workstack.append((3,a3.iloc[i]['dep'],stack3[i],i+1))
        for i in range(len(stack4)):
            if stack4[i] == count:
                workstack.append((4,a4.iloc[i]['dep'],stack4[i],i+1))
        count += 1
            
print(workstack)
# Ist = course number
# 2nd = dependency on lecture no.
# 3rd = no. of days left
# 4th = assignment number

[(1, 2, 4, 1), (1, 4, 5, 2)]


In [43]:
tasks = []
for i,j,k,m in workstack:
    if k==0:
        # do task immediately
        seq = cdf.iloc[i-1]['videos']['status']
        for l in range(j):
            if seq[l] == 'No':
                tasks.append((i,l+1,m))
        print(seq)
print(tasks)


# fill the tasks list
for i,j,k,m in workstack:
    seq = cdf.iloc[i-1]['videos']['status']
    for l in range(j):
        if seq[l] == 'No':
            tasks.append((i,l+1,m))
print(tasks)
# tasks 1st element = course number
# tasks 2nd element = lecture number
# tasks 3rd element = Assignmet number

numoftasks = int(input('Enter no. of tasks: '))
tasks = tasks[:numoftasks+1]
if len(tasks)==0:
    print('Hurray! No work!')
else:
    print(tasks)

[]


IndexError: list index out of range

In [24]:
def watch_video(conn, course_num, lecture_num):
    cur = conn.cursor()
    cur.execute('UPDATE Vid'+str(course_num)+' SET status="YES" WHERE ID=?',(lecture_num,))
    conn.commit()
    print('Watched: Course',course_num,'Lecture',lecture_num)

In [44]:
def do_assignment(conn, course_num, ass_num):
    cur = conn.cursor()
    cur.execute('UPDATE Ass'+str(course_num)+' SET status="YES" WHERE assID=?',(ass_num,))
    conn.commit()
    print('Done: Course',course_num,'Assignment',ass_num)

In [25]:
#watch_video(conn, 1, 1)

Watched: Course 1 Lecture 1


In [45]:
#do_assignment(con, 1, 2)